In [ ]:
# The code was removed by Watson Studio for sharing.

##  Segmenting and Clustering Neighborhoods in Toronto Whose Postal Code Begins With the Letter M - First Section

### 1.1 Creating Notebook with the Basic Dependencies

In [1]:
import numpy as np #library of handle data in a vectoized manner
import pandas as pd # library for data analysis
import requests # library for web scraping

print ('Libraries imported')

Libraries imported


### 1.2 Scraping Web Page Process Using BeautifulSoup

In [2]:
# installing Beautiful Soup for scraping data
# https://pypi.python.org/pypi/beautifulsoup4

# or download file
# http://beautiful-soup-4
# and unzip it in the same directory as this file

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
import json
import xml


print('BeautifulSoup and csv imported.')


BeautifulSoup and csv imported.


In [3]:
# Ignore SSL Certificate Errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL Certificate Errors Ignored')

SSL Certificate Errors Ignored


In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#print(source)

soup = BeautifulSoup(source, 'lxml')
#print(soup)

#print(soup.prettify())
print('Soup Ready')

Soup Ready


In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
#table
#print(table)

In [6]:
#from time import sleep
#sleep(3)

table_rows = table.find_all('tr')
#table rows

In [7]:
data=[]

for row in table_rows:
    data.append([t.text.strip() for t in row.find_all ('td')])
    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])    
df = df[~df['PostalCode'].isnull()] #to weed out bad rows

print(df.head(10))
print('****')
print(df.tail(10))

   PostalCode           Borough     Neighbourhood
1         M1A      Not assigned      Not assigned
2         M2A      Not assigned      Not assigned
3         M3A        North York         Parkwoods
4         M4A        North York  Victoria Village
5         M5A  Downtown Toronto      Harbourfront
6         M5A  Downtown Toronto       Regent Park
7         M6A        North York  Lawrence Heights
8         M6A        North York    Lawrence Manor
9         M7A      Queen's Park      Not assigned
10        M8A      Not assigned      Not assigned
****
    PostalCode       Borough             Neighbourhood
279        M4Z  Not assigned              Not assigned
280        M5Z  Not assigned              Not assigned
281        M6Z  Not assigned              Not assigned
282        M7Z  Not assigned              Not assigned
283        M8Z     Etobicoke  Kingsway Park South West
284        M8Z     Etobicoke                 Mimico NW
285        M8Z     Etobicoke        The Queensway West
286  

### 1.3.1 Process of Transforming Data into Pandas Dataframe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 3 columns):
PostalCode       288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [9]:
df.shape

(288, 3)

### 1.3.2 Dataframe With Unprocessed Data Established

In [10]:
import pandas
import requests
from bs4 import BeautifulSoup

website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text, 'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data=[]
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood']) 
df = df[~df['PostalCode'].isnull()] #to filter out bad rows
df.head(15)



,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### 1.3.3 Drop Rows Where Postal Code Does Not Have as Assigned Burough

In [11]:
df.drop(df[df['Borough']=="Not assigned"].index, axis=0, inplace=True)

### 1.3.4 Reindex Dataframe

In [12]:
df1 = df.reset_index()
df1.head(10)

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights
5,8,M6A,North York,Lawrence Manor
6,9,M7A,Queen's Park,Not assigned
7,11,M9A,Etobicoke,Islington Avenue
8,12,M1B,Scarborough,Rouge
9,13,M1B,Scarborough,Malvern


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index            211 non-null int64
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [14]:
df1.shape

(211, 4)

### 1.3.5 Combine multi neighborhood postal codes

In [25]:
df2 = df1.groupby('PostalCode').agg(lambda x: ','.join(x))

df2.head(10)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [18]:
df2.shape


(103, 2)

### 1.3.6 For Boroughs That Have No Assigned Neighborhoods, Assign The Borough Name to the Neighborhood Section

In [20]:
df2.loc[df2['Neighbourhood']=='Not assigned','Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.head(10)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


In [27]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.3.7 Remove Repeating Borough Names 

In [40]:
df3['Borough']=df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [37]:
df3.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [38]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [39]:
df3.shape

(103, 3)

### This Section Submitted to GetHub To Cover First Question of Assignment